In [1]:
import pickle
import gc
from datetime import datetime
from os import mkdir
from os.path import join, exists

import numpy as np
import pandas as pd
import scipy as sp

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

from sklearn import metrics

In [2]:
train_f = pd.read_pickle('../dataset/train/train_all_feature.pkl')
train_f.shape

(879539, 493)

In [3]:
test_f = pd.read_pickle('../dataset/train/test_all_feature.pkl')
test_f.shape

(215287, 493)

In [4]:
model_path = '../dataset/lightgbmmodel.pkl'

In [5]:
old_model_path = '../dataset/old_lightgbmmodel.pkl'

In [6]:
luck = 123456

In [7]:
model = LGBMClassifier(nthread=4, silent=False, objective= 'binary', seed=luck)

In [8]:
parameters = {
#     'gamma': [0.05],
    #     'n_estimators': [800],
    'max_depth': [2, 6, 12],
#     'max_depth': [12],
#     'num_leaves': range(30, 50, 4), 

#     'learning_rate': [0.15, 0.3],
#         'subsample': [0.9],
#         'colsample_bytree': [0.9],
    #     'reg_alpha': [0, 1, 5],
    #     'reg_lambda': [0, 1, 4],
}

In [9]:
# select_columns = pickle.load(open('select_columns.pkl', 'rb'))

In [10]:
use_columns = [
    x for x in train_f.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid',
        'orderlabel', 'index'
    ]
]

In [11]:
cv = model_selection.ShuffleSplit(n_splits=1, test_size=0.2, random_state=luck)
clf = model_selection.GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=True)

In [12]:
clf_feture = train_f.sample(1000, random_state=luck)

In [13]:
%time clf.fit(clf_feture[use_columns], clf_feture['orderlabel'])

Fitting 1 folds for each of 3 candidates, totalling 3 fits
CPU times: user 271 ms, sys: 47.1 ms, total: 318 ms
Wall time: 863 ms


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.6s finished


GridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=123456, test_size=0.2, train_size=None),
       error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=4,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=123456, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [2, 6, 12]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=True)

In [14]:
clf.best_score_

0.96499999999999997

In [15]:
clf.best_params_

{'max_depth': 2}

In [16]:
# # clf.best_params_['n_estimators'] = 
clf.best_params_['max_depth'] = 6
# clf.best_params_['reg_lambda'] = 0.5
clf.best_params_['subsample'] = 0.8
clf.best_params_['colsample_bytree'] = 0.8

clf.best_params_['colsample_bylevel'] = 0.7

# clf.best_params_['n_estimators'] = 1000
clf.best_params_['learning_rate'] = 0.1
clf.best_params_['num_leaves'] = 56


# clf.best_params_['reg_alpha'] = 1
# clf.best_params_['min_data_in_leaf'] = 3

# clf.best_params_['min_child_weight'] = 1
clf.best_params_['lambda_l2'] = 10
clf.best_params_['tree_method'] = 'exact'
clf.best_params_['metric'] = 'binary_logloss'



In [17]:
params = {
    'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'binary_logloss',
        'metric': 'auc',
    'min_child_weight': 1.5,
#     'num_leaves': 2 ** 7,
    'max_depth':6,
      'num_leaves':36,
#     'lambda_l2': 10,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
#     'colsample_bylevel': 0.7,
    'learning_rate': 0.05,
#     'tree_method': 'exact',
    'subsample_freq': 10,
#     'bagging_freq':10,
}

In [18]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=1300,
    **params)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.5, learning_rate=0.05,
        max_bin=255, max_depth=6, metric='auc', min_child_samples=10,
        min_child_weight=1.5, min_split_gain=0, n_estimators=1300,
        nthread=4, num_leaves=36, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=123456, silent=False, subsample=0.5,
        subsample_for_bin=50000, subsample_freq=10)

In [19]:
# categorical_feature=['order_' + x for x in  ['roomservice_8','roomservice_4','roomservice_3','roomtag_1']]

In [20]:
model.fit(
    train_f[use_columns],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns],
    train_f['orderlabel']),
             (test_f[use_columns],
    test_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
    early_stopping_rounds=100,
)

[1]	valid_0's auc: 0.886706	valid_1's auc: 0.887076
Train until valid scores didn't improve in 100 rounds.
[2]	valid_0's auc: 0.895766	valid_1's auc: 0.895556
[3]	valid_0's auc: 0.898251	valid_1's auc: 0.896667
[4]	valid_0's auc: 0.901199	valid_1's auc: 0.899835
[5]	valid_0's auc: 0.901941	valid_1's auc: 0.900851
[6]	valid_0's auc: 0.902272	valid_1's auc: 0.90141
[7]	valid_0's auc: 0.904301	valid_1's auc: 0.903968
[8]	valid_0's auc: 0.907959	valid_1's auc: 0.906495
[9]	valid_0's auc: 0.909047	valid_1's auc: 0.907557
[10]	valid_0's auc: 0.909203	valid_1's auc: 0.907709
[11]	valid_0's auc: 0.90992	valid_1's auc: 0.908263
[12]	valid_0's auc: 0.909955	valid_1's auc: 0.908456
[13]	valid_0's auc: 0.909893	valid_1's auc: 0.908515
[14]	valid_0's auc: 0.910315	valid_1's auc: 0.908802
[15]	valid_0's auc: 0.91041	valid_1's auc: 0.908991
[16]	valid_0's auc: 0.910309	valid_1's auc: 0.90895
[17]	valid_0's auc: 0.910363	valid_1's auc: 0.90899
[18]	valid_0's auc: 0.910281	valid_1's auc: 0.908975
[19]	

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.5, learning_rate=0.05,
        max_bin=255, max_depth=6, metric='auc', min_child_samples=10,
        min_child_weight=1.5, min_split_gain=0, n_estimators=1300,
        nthread=4, num_leaves=36, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=123456, silent=False, subsample=0.5,
        subsample_for_bin=50000, subsample_freq=10)

In [21]:
pickle.dump(model, open(old_model_path, 'wb'))

In [22]:
def get_score(model, train_f):
    result = model.predict_proba(train_f[use_columns])[:, 1]
    train_f['prob'] = result
    train_x = train_f[['orderid', 'prob', 'orderlabel']]
    train_x.sort_values('prob', ascending=False, inplace=True)
    finall_result_x = train_x.drop_duplicates(['orderid'])
    train_score = finall_result_x.orderlabel.mean()
    print(train_score)
    return train_score

In [23]:
train_score = get_score(model, train_f)

0.498587397126


/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
test_score = get_score(model, test_f)

0.438288805114


/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
acu = model.score(test_f[use_columns], test_f['orderlabel'])
acu

0.974271553786341

In [26]:
# y_predict = model.predict_proba(train_features[use_columns])[:, 1]
# fpr, tpr, thresholds  = metrics.roc_curve(train_features['orderlabel'], y_predict, pos_label=1)
# auc_score = metrics.auc(fpr, tpr)
# auc_score

In [27]:
if not exists('models'):
    mkdir('models')
model_importance_path = join('models', datetime.now().strftime('%d-%H%M-importance.txt'))
model_importance_path

'models/09-0753-importance.txt'

In [28]:
importance_df = pd.DataFrame(model.feature_importances_, index=use_columns)

importance_df.sort_values(0, ascending=False, inplace=True)
importance_df.to_csv(model_importance_path)

In [29]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in clf.best_params_.items()])
parms

'colsample_bylevel: 0.7,subsample: 0.8,colsample_bytree: 0.8,num_leaves: 56,learning_rate: 0.1,metric: binary_logloss,max_depth: 6,lambda_l2: 10,tree_method: exact'

In [30]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in params.items()])
parms

'subsample_freq: 10,boosting_type: gbdt,num_leaves: 36,colsample_bytree: 0.5,subsample: 0.5,min_child_weight: 1.5,metric: auc,max_depth: 6,learning_rate: 0.05'

In [31]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.best_score['valid_0']['auc'],
    'model_test',
    model.best_score['valid_1']['auc'],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    'shape', train_f.shape,
    end='\n',
    sep='  ')

09-07-53:  online  ??  test_score  0.438288805114  train_score  0.498587397126  model_train  0.959781127214  model_test  0.932979011974  accu  0.974271553786  n  800  params  subsample_freq: 10,boosting_type: gbdt,num_leaves: 36,colsample_bytree: 0.5,subsample: 0.5,min_child_weight: 1.5,metric: auc,max_depth: 6,learning_rate: 0.05  shape  (879539, 494)


In [32]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.best_score['valid_0']['auc'],
    'model_test',
    model.best_score['valid_1']['auc'],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    'shape', train_f.shape,
    end='\n',
    sep='  ', file=open('lgb_result.txt', 'a+'))

In [33]:
del train_f, test_f
gc.collect()

88

In [34]:
train_f = pd.read_pickle('../dataset/train/all_feature.pkl')
train_f.shape

(1094826, 493)

In [35]:
result = model.predict_proba(train_f[use_columns])[:, 1]

In [36]:
train_f['predict'] = result

In [37]:
n_est = model.best_iteration  - 200 if model.best_iteration > 0 else 1100

In [38]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=n_est,
    **params)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.5, learning_rate=0.05,
        max_bin=255, max_depth=6, metric='auc', min_child_samples=10,
        min_child_weight=1.5, min_split_gain=0, n_estimators=1000,
        nthread=4, num_leaves=36, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=123456, silent=False, subsample=0.5,
        subsample_for_bin=50000, subsample_freq=10)

In [39]:
# categorical_feature=['order_' + x for x in  ['roomservice_8','roomservice_4','roomservice_3','roomtag_1']]

In [43]:
model.fit(
    train_f[use_columns+['predict']],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns+['predict']],
    train_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
)

[1]	valid_0's auc: 0.887741
[2]	valid_0's auc: 0.897361
[3]	valid_0's auc: 0.903269
[4]	valid_0's auc: 0.93405
[5]	valid_0's auc: 0.939531
[6]	valid_0's auc: 0.938683
[7]	valid_0's auc: 0.94172
[8]	valid_0's auc: 0.94359
[9]	valid_0's auc: 0.942474
[10]	valid_0's auc: 0.941605
[11]	valid_0's auc: 0.940654
[12]	valid_0's auc: 0.943324
[13]	valid_0's auc: 0.942479
[14]	valid_0's auc: 0.941619
[15]	valid_0's auc: 0.940777
[16]	valid_0's auc: 0.942692
[17]	valid_0's auc: 0.941962
[18]	valid_0's auc: 0.94114
[19]	valid_0's auc: 0.942941
[20]	valid_0's auc: 0.942289
[21]	valid_0's auc: 0.941774
[22]	valid_0's auc: 0.9412
[23]	valid_0's auc: 0.940561
[24]	valid_0's auc: 0.940375
[25]	valid_0's auc: 0.939823
[26]	valid_0's auc: 0.93928
[27]	valid_0's auc: 0.938855
[28]	valid_0's auc: 0.938317
[29]	valid_0's auc: 0.937919
[30]	valid_0's auc: 0.939881
[31]	valid_0's auc: 0.941373
[32]	valid_0's auc: 0.942689
[33]	valid_0's auc: 0.942315
[34]	valid_0's auc: 0.941948
[35]	valid_0's auc: 0.941584
[

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.5, learning_rate=0.05,
        max_bin=255, max_depth=6, metric='auc', min_child_samples=10,
        min_child_weight=1.5, min_split_gain=0, n_estimators=1000,
        nthread=4, num_leaves=36, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=123456, silent=False, subsample=0.5,
        subsample_for_bin=50000, subsample_freq=10)

In [44]:
pickle.dump(model, open(model_path, 'wb'))

In [ ]:
# clf1 =  LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=42, max_depth=6)
# clf2 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=32, max_depth=5)
# clf3 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=27, max_depth=3)
# lr = LogisticRegression()
# sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3], 
#                           meta_classifier=lr)

# model = sclf